In [81]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from tqdm.notebook import tqdm as tn

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
RANDOM_STATE = 42

In [2]:
!wget https://github.com/andrii0yerko/INT20H-2022-Hackathon/blob/main/data/kfold.pkl?raw=true -O kfold.pkl
kf = pd.read_pickle('kfold.pkl')[['train_idx', 'test_idx']].to_records(index=False)

In [27]:
cb_test = pd.read_pickle('/kaggle/input/int20h-qual-catboost/catboost_test_meta.pkl').rename(columns={'model_0': 'catboost'})
cb_train = pd.read_pickle('/kaggle/input/int20h-qual-catboost/catboost_train_meta.pkl').rename(columns={'model_0': 'catboost'})

In [83]:
rf_test = pd.read_csv('/kaggle/input/random-forest-k-fold-test/oof_train_rf.csv').rename(columns={'Predicted': 'rf'})
rf_train = pd.read_csv('/kaggle/input/random-forest-k-fold-test/oof_test_rf.csv').rename(columns={'Predicted': 'rf'})

In [14]:
lw_test = pd.read_csv('/kaggle/input/lgbm-wide-best/oof_test.csv').rename(columns={'Predicted': 'lgbm-wide'})
lw_train = pd.read_csv('/kaggle/input/lgbm-wide-best/oof_train.csv').rename(columns={'Predicted': 'lgbm-wide'})

ll_test = pd.read_csv('/kaggle/input/lgbm-long/oof_test.csv').rename(columns={'Predicted': 'lgbm-long'})
ll_train = pd.read_csv('/kaggle/input/lgbm-long/oof_train.csv').rename(columns={'Predicted': 'lgbm-long'})

In [17]:
INPUT_DIR = '/kaggle/input/techuklon-int20h'

train = pd.read_csv(f'{INPUT_DIR}/train.csv')
test = pd.read_csv(f'{INPUT_DIR}/test.csv')

In [89]:
from functools import reduce
# [train.loc[train.Week == 0, ['Id', 'target']]
meta_train = reduce(lambda x, y: pd.merge(x, y, on='Id'), [lw_train, ll_train, cb_train])
meta_test = reduce(lambda x, y: pd.merge(x, y, on='Id'), [lw_test, ll_test, cb_test])
meta_train.to_csv('meta_train.csv')
meta_test.to_csv('meta_test.csv')

In [32]:
meta_train[(meta_train['catboost'] > meta_train['lgbm-wide']) & (meta_train['target']==1)]